In [2]:
import time
import pandas as pd
from review_scraper import AmazonScraper
import os

In [3]:
#questa cella scrapa le recensioni

amz_scraper = AmazonScraper()

products_url = {"echo_dot.csv":"https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=1",
                "echo_4thgen.csv":"https://www.amazon.com/Echo-4th-Gen/product-reviews/B07XKF5RM3/ref=cm_cr_arp_d_viewopt_rvwer?ie=UTF8&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=1"
                }

#se hai bisogno di capire come trovare la url giusta scrivimi

for product in products_url:
    reviews = []
    for page in range(1, 50):
        response = amz_scraper.scrapeReviews(url=products_url[product], page_num=page, filter_by="helpful", star="critical")
        reviews.extend(response)
        time.sleep(1)

    df = pd.DataFrame(reviews)

    #######################################################
    #pulisce i dati

    ratings = []
    feedbacks = []

    for score in df["rating"]:
        try:
            value = int(score[0])
        except TypeError:
            print("Errore sul valore:", score)
        if value <= 2:
            feedbacks.append(0)
        elif value >= 3:
            feedbacks.append(1)

        ratings.append(value)

    df["rating"] = ratings
    df["feedback"] = feedbacks
    df.drop(["review_title", "username", "profile_url", "verified_purchase"], axis=1, inplace=True)
    df.rename({"product_name":"variation", "comment":"verified_reviews"}, inplace=True, axis=1)

    #######################################################

    df.to_csv(f"scraped_data/{product}", sep="\t", index = False) #salva

Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=1...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=2...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=3...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=4...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&reviewerType=avp_only_reviews&filterByStar=critical&pageNumber=5...
Processing https://www.amazon.com/Echo-Dot/product-reviews/B07FZ8S74R?reviewerType=all_reviews&sortBy=helpful&

In [4]:
#################################
#unisce tutti i df estratti in uno unico
df_list = []

for path in os.listdir("scraped_data"):
    df = pd.read_csv(f"scraped_data/{path}", sep = "\t")
    df_list.append(df)

total_df = pd.concat(df_list)

total_df

,variation,verified_reviews,rating,date,feedback
0,Echo 4th Gen,QUICK SUMMARY:---------------------------The 4...,3,"October 3, 2021",1
1,Echo 4th Gen,I will start off on a good note: The sound is...,3,"October 17, 2022",1
2,Echo 4th Gen,"I was super excited to get these, I’ve had an ...",1,"August 23, 2022",0
3,Echo 4th Gen,[Edit] This is my second time to edit this rev...,1,"January 14, 2021",0
4,Echo 4th Gen,Update October 11—-DO NOT BUY. AMOAZON WILL FO...,1,"July 8, 2022",0
...,...,...,...,...,...
485,Echo Dot,this device is kind of lost on me. I don't rea...,3,"February 3, 2021",1
486,Echo Dot,They didn't have any Jamaican or Rasta voices,3,"April 8, 2022",1
487,Echo Dot,I have newer Alexa's but I ordered this one be...,2,"June 8, 2022",0
488,Echo Dot,Having issues with voice recognition. Purchase...,2,"January 16, 2020",0


In [5]:
total_df["rating"].value_counts()

3    730
2    174
1     76
Name: rating, dtype: int64

In [ ]:
# Ci sono alcune reviews in spagnolo (le recensioni vengono amazon.com, quindi america), si può valutare di tradurle.

In [6]:
total_df.to_csv("test_set.csv", sep = "\t") #salva il df totale come test set